In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
all_data_with_freq =  pd.read_csv("/localdisk/home/s1533486/Synovium_PBMCs/immcantation/analysis_ready/all_data_with_muts_by_frequency.tab", sep='\t')
all_data_with_muts =  pd.read_csv("/localdisk/home/s1533486/Synovium_PBMCs/immcantation/analysis_ready/all_data_with_muts.tab", sep='\t')
#Dropping patient F due to monoclonal repertoire 
all_data_with_muts = all_data_with_muts[all_data_with_muts['PATIENT'] != 'F']
all_data_with_freq = all_data_with_freq[all_data_with_freq['PATIENT'] != 'F']
RAbaseline_data_freq = pd.read_csv("/localdisk/home/s1533486/RAbaseline/clones/RAbaseline_data_with_muts_by_frequency.tab", sep='\t')
RAbaseline_data_muts = pd.read_csv("/localdisk/home/s1533486/RAbaseline/clones/RAbaseline_data_with_muts.tab", sep='\t')
frames = [RAbaseline_data_freq,all_data_with_freq]
mut_frames = [RAbaseline_data_muts, all_data_with_muts]
full_dataset_freq = pd.concat(frames)
full_dataset_muts = pd.concat(mut_frames)

In [2]:
# Adding mutational_status column to patient data 
full_dataset_muts.loc[full_dataset_muts['MU_COUNT']  < 5, 'MUT_STATUS'] = 'Hypomutated'
full_dataset_muts.loc[full_dataset_muts['MU_COUNT']  >= 5, 'MUT_STATUS'] = 'Hypermutated'
full_dataset_muts['PATIENT_LOCATION'] = full_dataset_muts['PATIENT'] + '-' + full_dataset_muts['LOCATION']

In [3]:
# How many patients use IGHV4-34?
patients = full_dataset_muts[full_dataset_muts['LOCATION'] == 'Periphery']
ighv4_34 = patients[patients['V_CALL_GENOTYPED'].astype(str).str.contains("IGHV4-34")]
print(len(patients['PATIENT'].unique()))
print(len(ighv4_34['PATIENT'].unique()))

26
10


In [4]:
patients = full_dataset_muts[full_dataset_muts['LOCATION'] == 'Periphery']
ighv4_34 = patients[patients['V_CALL_GENOTYPED'].astype(str).str.contains("IGHV4-34")]
ighv434 = ighv4_34.groupby(['PATIENT','V_CALL_GENOTYPED'])['MUT_STATUS'].value_counts().reset_index(name='counts')
ighv434 = ighv434.groupby(['PATIENT','MUT_STATUS'])['counts'].sum().reset_index(name='counts')
counts = []
for n in range(len(ighv434)):
    state_reads = len(full_dataset_muts.loc[(full_dataset_muts['PATIENT'] == ighv434.iloc[n]['PATIENT']) & (full_dataset_muts['MUT_STATUS'] == ighv434.iloc[n]['MUT_STATUS'])])
    new_count = ((ighv434.iloc[n]['counts']/state_reads)*100).astype(float)
    counts.append(new_count)
ighv434['counts'] = counts
# Mean and confidence interval of IGHV4-34 hypomutated sequences
import statsmodels.stats.api as sms
import statistics
ci = sms.DescrStatsW(ighv434[ighv434['MUT_STATUS'] == 'Hypomutated']['counts']).tconfint_mean(alpha=0.05)
mean = ighv434[ighv434['MUT_STATUS'] == 'Hypomutated']['counts'].mean()
std = statistics.stdev(ighv434.loc[ighv434['MUT_STATUS'] == 'Hypomutated','counts'])
print('Hypomutated IGHV4-34 usage mean usage (%) = ', mean, ', 95% CI [',ci[0].round(3), ',',ci[1].round(3), '], SD =', round(std,3))

ci = sms.DescrStatsW(ighv434[ighv434['MUT_STATUS'] == 'Hypermutated']['counts']).tconfint_mean(alpha=0.05)
std = statistics.stdev(ighv434.loc[ighv434['MUT_STATUS'] == 'Hypermutated','counts'])
mean = ighv434[ighv434['MUT_STATUS'] == 'Hypermutated']['counts'].mean()
print('Hypermutated IGHV4-34 usage mean usage (%) = ', mean, ', 95% CI [',ci[0].round(3), ',',ci[1].round(3), '], SD =', round(std,3))

Hypomutated IGHV4-34 usage mean usage (%) =  2.3555907678024073 , 95% CI [ 0.318 , 4.393 ], SD = 2.65
Hypermutated IGHV4-34 usage mean usage (%) =  3.7757502356961656 , 95% CI [ 1.774 , 5.777 ], SD = 2.394


In [5]:
# As above but for healthy donors
donors = full_dataset_muts[full_dataset_muts['LOCATION'] == 'Healthy_Periphery']
ighv4_34 = donors[donors['V_CALL_GENOTYPED'].astype(str).str.contains("IGHV4-34")]
print(len(donors['PATIENT'].unique()))
print(len(ighv4_34['PATIENT'].unique()))

16
1


In [6]:
#IGHV4-34 IN HD P
ighv434 = ighv4_34.groupby(['PATIENT','V_CALL_GENOTYPED'])['MUT_STATUS'].value_counts().reset_index(name='counts')
ighv434 = ighv434.groupby(['PATIENT','MUT_STATUS'])['counts'].sum().reset_index(name='counts')
counts = []
for n in range(len(ighv434)):
    state_reads = len(full_dataset_muts.loc[(full_dataset_muts['PATIENT'] == ighv434.iloc[n]['PATIENT']) & (full_dataset_muts['MUT_STATUS'] == ighv434.iloc[n]['MUT_STATUS'])])
    new_count = ((ighv434.iloc[n]['counts']/state_reads)*100).astype(float)
    counts.append(new_count)
ighv434['counts'] = counts
ighv434
#counts = %

,PATIENT,MUT_STATUS,counts
0,B6844,Hypermutated,0.941677
1,B6844,Hypomutated,0.625000
